In [1]:
## Data scrapping prpm
from PyClasses.Scrapers.PRPMScraper import PRPMScraper
PRPMscrap = PRPMScraper()

## find sinonim still buggy (will work if got time)
result = PRPMscrap.findWordMetaData('selain')
print(result)

[<b>Bersinonim dengan </b>]
{'selain': {'meanings': ['1. hanya, kecuali: ia tidak dapat berbuat apa-apa ~ membiarkan sahaja; 2. di samping, bersama-sama dgn: ~ ceramah, diadakan juga pembacaan sajak;', ', = lain drpd 1 di samping: ~ berlatih kawad, mereka belajar undang-undang juga. 2 kecuali; hanya: Emaknya tidak dpt berbuat apa-apa, ~ menangis.'], 'sinonim': ['berbeza']}}


In [21]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from datetime import datetime, timedelta

from bs4 import BeautifulSoup

import os
os.environ['DISPLAY'] = ':0'

# BeritaHarina max page = 20 only (as of 5 Dec 2024)
class BHarianScraper:
    __month_mapping = {
        "Jan": 1, "Feb": 2, "Mac": 3, "Apr": 4, "Mei": 5, "Jun": 6,
        "Jul": 7, "Ogs": 8, "Sep": 9, "Okt": 10, "Nov": 11, "Dis": 12
    }
    
    def __init__(self,category = None):
        self.baseUrl = 'https://www.bharian.com.my/'
        self.category = category
        self.options = Options()
        self.options.add_argument("--headless")  # Run in headless mode
        self.options.add_argument("--disable-gpu")  # Disable GPU acceleration
        self.options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems
        self.options.add_argument("--incognito") 
        self.options.add_argument('user-agent=Mozilla/5.1 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')
        self.service = Service(ChromeDriverManager().install())
        self.driver = None
        self.relLinkToArticle_all = []
        self.fromDate = None
        self.toDateTime = None
    
    def scrapArticleLinks(self):
        # #if category not specified raise exceptions
        # if category == None and self.category == None:
        #     raise CategoryError(category,f"Category is invalid. Category=\'{self.category}\'. Please specify category in the form of url appending string.")

        # #Assign new cat if new
        # self.category = category if category is not None else self.category
        # self.fromDate = fromDate
        # self.toDateTime = datetime.now()
        self.relLinkToArticle_all = []
        article_teasers=[]
        #Web Crawl Begins
        for i in range (0,21):
            try:
                self.driver = webdriver.Chrome(service=self.service, options=self.options)
                self.driver.get(f"{self.baseUrl}{self.category}?page={i}")
                self.driver.implicitly_wait(10)

                try:
                    # Wait for the div with class article-teaser to appear
                    WebDriverWait(self.driver, 20).until(
                        EC.presence_of_element_located((By.CLASS_NAME, 'article-teaser'))
                    )
                    print("Article teaser found.")
                except Exception as e:
                    print(f"Error: {e}")
                    return None
                # Get the page HTML Text 
                page_source = self.driver.page_source
                soup = BeautifulSoup(page_source, "html.parser")
                article_teasers = soup.find_all(
                    lambda tag: tag.name == 'div' 
                    and 'article-teaser' in tag.get('class', []) 
                    and 'd-block' not in tag.get('class', [])
                )
                # for content in article_teasers:
                #     print(content.prettify())
                # print(len(article_teasers))
            finally:
                self.driver.quit()
                print("driverQuit")
    
            # scrap all info
            for teaser in article_teasers:
                # get article create time
                span_PublishTimeInfo = teaser.find('span', class_='created-ago')
                timePublish_text = span_PublishTimeInfo.text.strip()
    
                # get article link
                aTag = teaser.find('a',class_='d-flex article listing mb-3 pb-3')
                relLinkToArticle = aTag['href']
                
                # check time within range
                timePublish_datetime = self.convertTimeStringToDateTime(timePublish_text,toDateTime=self.toDateTime)
                if(timePublish_datetime > self.fromDate):
                    self.relLinkToArticle_all.append(relLinkToArticle)
                else:
                    return self.relLinkToArticle_all
        


    def convertTimeStringToDateTime(self, timePublish_text,toDateTime):
        if '@' in timePublish_text:
            pratitonedTimeTxt = timePublish_text.split()
            month = BHarianScraper.__month_mapping[pratitonedTimeTxt[0]]  # Convert "Dis" to 12
            day = int(pratitonedTimeTxt[1].strip(','))
            year = int(pratitonedTimeTxt[2])
            time_str = pratitonedTimeTxt[4]
            time_format = "%I:%M%p"
            return (datetime.strptime(f"{year}-{month}-{day} {time_str}", "%Y-%m-%d %I:%M%p"))
        elif 'sejam' in timePublish_text:
            return (toDateTime - timedelta(hours=1))
        elif 'jam' in timePublish_text:
            hoursAgoInt = int(timePublish_text.split()[0])
            return (toDateTime - timedelta(hours=hoursAgoInt))
        # minit before
        else:
            minutesAgoInt = int(timePublish_text.split()[0])
            return (toDateTime - timedelta(minutes=minutesAgoInt))
            

    def scrapArticles(self,fromDate, category = None):
        #if category not specified raise exceptions
        if category == None and self.category == None:
            raise CategoryError(category,f"Category is invalid. Category=\'{self.category}\'. Please specify category in the form of url appending string.")

        #Assign new cat if new
        self.category = category if category is not None else self.category
        self.fromDate = fromDate
        self.toDateTime = datetime.now()
        self.relLinkToArticle_all = self.scrapArticleLinks()

    def scrapOneArticle(self,url):
        soup = None
        try:
            self.driver = webdriver.Chrome(service=self.service, options=self.options)
            self.driver.get(url)
            self.driver.implicitly_wait(10)
            # try:
            #     # Wait for the div with class article-teaser to appear
            #     WebDriverWait(self.driver, 20).until(
            #         EC.presence_of_element_located((By.CLASS_NAME, 'article-teaser'))
            #     )
            #     print("Article teaser found.")
            # except Exception as e:
            #     print(f"Error: {e}")
            #     return None
            # Get the page HTML Text 
            page_source = self.driver.page_source
            soup = BeautifulSoup(page_source, "html.parser")
            
        finally:
            self.driver.quit()
            print("driverQuit Article")

        #get Article Title
        titleEle = soup.find('span', class_='d-inline-block mr-1')
        title = titleEle.get_text()

        author_n_dateTimeEle = soup.find('div',class_='article-meta mb-2 mb-lg-0 d-flex align-items-center').find('div')
        
        #get author name
        author_aTag = author_n_dateTimeEle.find('a')
        author = author_aTag.get_text()
        
        #get author link in BH
        authorRelativeLink = author_aTag['href']
        author_aTag.decompose()
        
        #get author email
        author_email = ''
        email_aTag = author_n_dateTimeEle.find('a')
        author_email = email_aTag.get_text()
        email_aTag.decompose()
        
        # get date time 
        author_n_dateTimeEle.find('span').decompose()
        publishTime_txt = author_n_dateTimeEle.get_text().strip()
        print(publishTime_txt)
        
        # get article brief
        articleBrief = soup.find('figcaption',class_='p-2').get_text()

        # get article content
        articleContent_soup = soup.find('div',class_='dable-content-wrapper')
        articleContents = self.getArticleContents(articleContent_soup)
        print(articleContents)
        
    def getArticleContents(self,soup):
        #remove uncessary div if any
        contents = []
        divToRm = soup.find_all('div',class_='google-anno-skip google-anno-sc')
        for div in divToRm:
            div.decompose()
        all_para = soup.find_all('p')
        filtered_para_all = [p for p in all_para if p.get_text()]
        for p in filtered_para_all:
            p.string = p.get_text().replace('\xa0', ' ')
            contents.append(p.get_text())
        return contents
            
        
        
class CategoryError(Exception):
    def __init__(self, category, message="Category is invalid."):
        self.category = category
        self.message = message
        super().__init__(self.message)     

bharianScraper = BHarianScraper()
# fromDate = datetime(2024,12,4)
# print(fromDate)
# articlesData = bharianScraper.scrapArticles(fromDate=fromDate,category='sukan/bola')
# print(len(bharianScraper.relLinkToArticle_all))
bharianScraper.scrapOneArticle('https://www.bharian.com.my/sukan/bola/2024/12/1332562/hamidin-lepas-jawatan-presiden-fam')

driverQuit Article
Disember 4, 2024 @ 12:42pm
Hamidin tidak akan pertahankan jawatan Presiden FAM pada kongres badan induk itu Februari depan.
['KUALA LUMPUR: Presiden Persatuan Bolasepak Malaysia (FAM), Tan Sri Hamidin Mohd Amin tidak akan mempertahankan jawatannya pada kongres pemilihan badan induk itu, tahun depan.', 'Perkara itu diumumkan Hamidin pada sidang media selepas Mesyuarat Ke-13 Jawatankuasa Eksekutif FAM bagi Penggal 2021-2025 di Wisma FAM, hari ini.', 'Beliau menjelaskan keputusan itu diambil bagi memberi ruang kepada ahli gabungan memilih presiden baharu pada kongres pemilihan, 15 Februari depan.', '"Saya dengan rasa rendah diri ingin memaklumkan bahawa saya tidak akan mempertahankan jawatan Presiden FAM bagi penggal akan datang," kata Hamidin.', 'Pada 2018, Hamidin dipilih menjadi Presiden FAM bagi mengisi kekosongan yang ditinggalkan Tunku Mahkota Ismail sebelum beliau menang tanpa bertanding pada kongres pemilihan bagi penggal 2021-2025.']


### Title
<span class="d-inline-block mr-1">Hamidin lepas jawatan Presiden FAM</span>

### Author, Author Link, DateTime
<div class="article-meta mb-2 mb-lg-0 d-flex align-items-center"><!----> 
	<div>
		<span>Oleh <a href="/authors/zulfaizan-mansor" target="_blank">Zulfaizan Mansor</a> - </span>
            Disember 4, 2024 @ 12:42pm
    <div>
		<a href="mailto:zulfaizan@mediaprima.com.my">zulfaizan@mediaprima.com.my</a>
		</div>
	</div>
</div>

### Images, Article Brief Info
<div class="field field-featured-image article-image-item">
    <figure>
        <img src="https://assets.bharian.com.my/images/articles/HAMIDIN_MOHD_AMIN_SIDANG_MEDIA_FAM_1733287320.jpg" alt="Hamidin tidak akan pertahankan jawatan Presiden FAM pada kongres badan induk itu Februari depan." class="img-fluid"> 
        <figcaption class="p-2">Hamidin tidak akan pertahankan jawatan Presiden FAM pada kongres badan induk itu Februari depan.
        </figcaption>
    </figure>
</div>

### Article Content
<div itemprop="articleBody" class="dable-content-wrapper">
    <p></p>
    <p>KUALA LUMPUR: Presiden Persatuan Bolasepak Malaysia (FAM), Tan Sri Hamidin Mohd Amin tidak akan mempertahankan jawatannya pada kongres pemilihan badan induk itu, tahun depan.
        <div aria-label="Tonton bola sepak secara langsung dalam talian" class="google-anno-skip google-anno-sc" data-google-interstitial="false" data-google-vignette="false" role="link" style="color-scheme: initial : ift: ortant; hyphenate-limit-chars: initial !important; hyphens: initial !importantitial !important; -webkit-ruby-position: initial !important; -webkit-tap-highlight-color: initialtart-end-radius: initial !important; border-start-start-radius: initial !important; bottom: initial !important; box-decoration-break: initial !important; box-shadow: none !important; box-sizing: initial !important; br initial !important; inset-block: initial !important; inset-inline: initial !important; interpolate-size: i t; -webkit-mask-box-image: initial !important; -webkit-print-color-adjust: initial !important; -webkit-rtl-ordering: initial !important; -webkit-ruby-position: initial !important; -webkit-tap-highlight-color: initial !important; -webkit-text-combine: initial !important; -webkit-text-decorations-in-effect: initial !important; -webkit-text-fill-color: unset !important; -webkit-text-security: initial !important; -webkit-text-stroke: initial !important; -webkit-user-drag: initial !important; white-space-collapse: initial !important; widows: initial !important; width: initial !important; will-change: initial !important; word-break: initial !important; word-spacing: initial !important; x: initial !important; y: initial !important; z-index: initial !important;">Tonton bola sepak secara langsung dalam talian</span>
        </div>
    </p>
    <p>Perkara itu diumumkan Hamidin pada sidang media selepas Mesyuarat Ke-13 Jawatankuasa Eksekutif FAM bagi Penggal 2021-2025 di Wisma FAM, hari ini.
    </p>
    <p>Beliau menjelaskan keputusan itu diambil bagi memberi ruang kepada ahli gabungan memilih presiden baharu pada kongres pemilihan, 15 Februari depan.
    </p>
    <p>"Saya dengan rasa rendah diri ingin memaklumkan bahawa&nbsp;saya tidak akan mempertahankan jawatan Presiden FAM bagi penggal akan datang," kata Hamidin.
    </p>
    <div class="related-listing"><div class="block block-article-related mt-5 mb-1"><div><div class="block-title-wrapper d-flex"><div class="block-title h2">Disyorkan</div> <!----></div></div> <div class="col-12"><div class="row"><div class="col-12"><ul><div class="block my-4"><li class="mb-2 article-thumbnails"><div class="content"><h5 class="field-title m-0"><a href="https://www.bharian.com.my/bisnes/korporat/2024/12/1332925/ceo-tmc-life-sciences-didapati-bersalah-atas-pertuduhan-salah-laku?utm_source=bh&amp;utm_medium=recsys" class="article listing">CEO TMC Life Sciences didapati bersalah atas pertuduhan salah laku</a></h5></div></li><li class="mb-2 article-thumbnails"><div class="content"><h5 class="field-title m-0"><a href="https://www.bharian.com.my/bisnes/auto/2024/12/1332913/bermaz-motor-lancar-mazda-cx-5-edisi-terhad-pamer-cx-60-dan-cx-80?utm_source=bh&amp;utm_medium=recsys" class="article listing">Bermaz Motor lancar Mazda CX-5 Edisi Terhad, pamer CX-60 dan CX-80</a></h5></div></li><li class="mb-2 article-thumbnails"><div class="content"><h5 class="field-title m-0"><a href="https://www.bharian.com.my/sukan/bola/2024/12/1332907/rainbow-devils-kecewa-man-united-tidak-pakai-jaket-pelangi?utm_source=bh&amp;utm_medium=recsys" class="article listing">Rainbow Devils kecewa Man United tidak pakai jaket pelangi</a></h5></div></li><li class="mb-2 article-thumbnails"><div class="content"><h5 class="field-title m-0"><a href="https://www.bharian.com.my/sukan/lain-lain/2024/12/1332909/malaysia-tempat-keempat-piala-remaja-asia-2024?utm_source=bh&amp;utm_medium=recsys" class="article listing">Malaysia tempat keempat Piala Remaja Asia 2024</a></h5></div></li></div></ul></div></div></div></div></div><div id="div-gpt-ad-1535527495770-0"></div>
    <p>Pada 2018, Hamidin dipilih menjadi Presiden FAM bagi mengisi kekosongan yang ditinggalkan Tunku Mahkota Ismail sebelum beliau menang tanpa bertanding pada kongres pemilihan bagi penggal 2021-2025.
    </p>
</div>

In [24]:
print (bharianScraper.relLinkToArticle_all[-1])

/sukan/bola/2024/12/1332419/mateta-hukum-kesilapan-benteng-ipswich
